In [ ]:
!pip install langchain
!pip install InstructorEmbedding
!pip install sentence-transformers==2.2.2
!pip install html2text
!pip install langchain-community
!pip install newspaper3k
!pip install chromadb
!pip install langchain-chroma langchain-text-splitters


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!pip install -q -U langchain langchain-community langchain-core
import os

from langchain_chroma import Chroma
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, NewsURLLoader
from langchain.chains import RetrievalQA

from langchain.document_loaders import TextLoader

from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 501.4/501.4 kB 31.3 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'langchain.chains'

In [ ]:
with open(r"/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 03 - RAG para documentos/CNN_Links.txt") as f:
    urls = [link.replace("\n", "") for link in set(f.readlines())]

loader = NewsURLLoader(urls=urls)
data = loader.load()
print("Primeira notícia: ", data[0])
print("\nSegunda notícia: ", data[1])


In [ ]:
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")

In [ ]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = '/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 03 - RAG para documentos/db'

## Here is the nmew embeddings being used
embedding = instructor_embeddings

#vectordb = Chroma.from_texts([], embedding=embedding, persist_directory=persist_directory)
contents = [item.page_content for item in data]
sources = [{'source_info': item.metadata['link']} for item in data]

# Initialize Chroma without specifying texts initially
vectordb = Chroma.from_texts(contents, embedding, metadatas = sources, persist_directory=persist_directory)

vectordb.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [ ]:
#Declaração do modelo de embedding
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
persist_directory = '/content/drive/MyDrive/Colab Notebooks/fine-tuning-fiap/Aula 03 - RAG para documentos/db'
embedding = instructor_embeddings

#Criação do objeto de gestão da vector db
vectordb = Chroma(embedding_function=embedding, persist_directory=persist_directory)

retriever = vectordb.as_retriever()
query = input("The first question:  ")
#Retorna os 3 primeiros itens relevantes para a pergunta do usuário
context = retriever.get_relevant_documents(query)[:1]

load INSTRUCTOR_Transformer
max_seq_length  512
The first question:  where in brazil there was a recent flood?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [ ]:
context

[Document(page_content='CNN —\n\nAt least 57 people have been killed and a further 373 are missing as heavy rain and flooding hit the Brazilian state of Rio Grande do Sul this week.\n\nAt least 74 others have been injured amid the series of catastrophic floods that has affected 281 municipalities, according to the latest figures shared by the civil defense Saturday.\n\nThe local government has declared a state of calamity in areas where more than 67,000 people are impacted. Almost 10,000 have been displaced and more than 4,500 are in temporary shelters, the civil defense said.\n\nAuthorities are closely monitoring dams that are not designed to handle such a high volume of water but said there is no imminent risk of failure.\n\nView of a flooded house at the Sarandi neighborhood in Porto Alegre, Rio Grande do Sul state, Brazil on May 3, 2024. Anselmo Cunha/AFP/Getty Images\n\nBrazilian President Luiz Inácio Lula da Silva met with local officials overseeing the relief efforts Thursday.\n